In [1]:
using LinearAlgebra
using SparseArrays
using Plots
using DifferentialEquations
using BenchmarkTools
using StaticArrays
gr()

Plots.GRBackend()

In [2]:
# Nx = 200
# Ny = 100
LeftX = 0
LeftY = 0
RightX = 10
RightY = 5
Lx = RightX-LeftX
Ly = RightY-LeftY
# hx = (RightX - LeftX)/Nx 
# hy = (RightY - LeftY)/Ny 
tStart = 0
tEnd = 4
FT = Float64

Float64

In [3]:
function coeffK(x, y, RightX, RightY)
    result = zeros(size(x))
    line_x = findall(x.<(RightX/2))[end][2]
    line_y = findall(y.<(RightY/2))[end][1]
    result[1:line_y, 1:line_x] .= 0.1
    result[line_y+1:end, 1:line_x] .= 0.4
    result[line_y+1:end, line_x+1:end] .= 0.7
    result[1:line_y, line_x+1:end] .= 1.0
    return result
end

function create2DLFVM(Nx, Ny, hx, hy, coeffFun, LeftX, LeftY, RightX, RightY)
    x_points = [j for j in LeftX:hx:RightX][2:end-1]
    y_points = [i for i in LeftY:hy:RightY][2:end-1]
    x = zeros((Ny-1, Nx-1))
    y = zeros((Ny-1, Nx-1))

    for j in 1:length(y_points)
        x[j, :] = x_points
    end

    for i in 1:length(x_points)
        y[:, i] = y_points
    end

    main_arr = ((1 / (hx * hx)) .* (coeffFun(x .- 0.5 * hx, y, RightX, RightY) .+ coeffFun(x .+ 0.5 * hx, y, RightX, RightY)) .+ (1 / (hy * hy)) .* (coeffFun(x, y .- 0.5 * hy, RightX, RightY) .+ coeffFun(x, y .+ 0.5 * hy, RightX, RightY)))
    main_arr = vec(main_arr')
    k3_arr = -(coeffFun(x, y .+ 0.5 * hy, RightX, RightY)) ./ (hy * hy)
    k3_arr = vec(k3_arr')
    k3_arr = k3_arr[1:(end - Nx + 1)]
    k1_arr = -(coeffFun(x .+ 0.5 * hx, y, RightX, RightY)) ./ (hx * hx)
    k1_arr[:, end] .= 0
    k1_arr = vec(k1_arr')
    k1_arr = k1_arr[1:(end - 1)]
    A = SparseArrays.spdiagm(0 => main_arr, -1 => k1_arr, 1 => k1_arr, (-Nx+1) => k3_arr, (Nx-1) => k3_arr)
    return A
end

# @code_warntype create2DLFVM(Nx, Ny, hx, hy, coeffK, LeftX, LeftY, RightX, RightY)
# @code_warntype coeffK(zeros((Ny-1, Nx-1)), zeros((Ny-1, Nx-1)), RightX, RightY)

create2DLFVM (generic function with 1 method)

In [4]:
function grid(Nx, Ny, Lx, Ly)
    points = Vector{SVector{2, Float64}}(undef, (Nx-1) * (Ny-1))
    index = 1
    for j in 1:(Ny-1)
        for i in 1:(Nx-1)
            points[index] = @SVector [i * Lx / Nx, j * Ly / Ny]
            index += 1
        end
    end
    return points
end
# @code_warntype grid(Nx, Ny, Lx, Ly)
# @btime grid(Nx, Ny, Lx, Ly)

function sourcefunc(points, t::FT, RightX, RightY)
    f_values = similar(points, Float64)  # Preallocate array for f values

    for (i, point) in enumerate(points)
        x, y = point[1], point[2]
        f_values[i] = sin(4*π*t) * (exp(-40.0 * (x - 0.25 * RightX)^2 - 40.0 * (y - 0.25 * RightY)^2) +
                                     exp(-40.0 * (x - 0.25 * RightX)^2 - 40.0 * (y - 0.75 * RightY)^2) +
                                     exp(-40.0 * (x - 0.75 * RightX)^2 - 40.0 * (y - 0.75 * RightY)^2) +
                                     exp(-40.0 * (x - 0.75 * RightX)^2 - 40.0 * (y - 0.25 * RightY)^2))
    end

    return f_values
end

# @code_warntype sourcefunc(points, 1.0, RightX, RightY)
# @btime sourcefunc(points, 1.0, RightX, RightY)

sourcefunc (generic function with 1 method)

In [5]:
# function RHS!(ddu, du, u, p, t)
#     A, sourcefunc, points, RightX, RightY = p
#     fLX = sourcefunc(points, t, RightX, RightY)
#     ddu .= -A*u .+ fLX
# end

# # @btime create2DLFVM(Nx, hx, hy, coeffK)
# # @code_warntype create2DLFVM(Nx, hx, hy, coeffK)
# A = create2DLFVM(Nx, Ny, hx, hy, coeffK, LeftX, LeftY, RightX, RightY)
# u0 = zeros((Nx-1)*(Ny-1), 1)
# du0 = u0
# tspan = (tStart, tEnd)
# points = grid(Nx, Ny, Lx, Ly)
# p = (; A, sourcefunc, points, RightX, RightY)

# problem = SecondOrderODEProblem(RHS!, du0, u0, tspan, p)
# solution = solve(problem, Midpoint())
# # @btime solve(problem, Tsit5())
# # @code_warntype solve(problem, Tsit5())

In [6]:
# # duEnd = solutions[1:((Nx-1)*(Ny-1))]
# uEnd = solution.u[end][((Nx-1)*(Ny-1)+1):end]
# @btime solution.u[end][((Nx-1)*(Ny-1)+1):end]

# # duEnd1 = reshape(duEnd, Nx-1, Ny-1)'
# uEnd1 = reshape(uEnd, Nx-1, Ny-1)
# @btime reshape(uEnd, Nx-1, Ny-1)

In [7]:
# t_values = solution.t

# animation = @animate for i in 1:length(t_values)
#     u_k = solution.u[i][((Nx-1)*(Ny-1)+1):end]
#     u_k1 = reshape(u_k, Nx-1, Ny-1)
#     heatmap(u_k1', c=:blues, aspect_ratio=:equal, xlabel="X", ylabel="Y", title="Time: $(t_values[i])")
# end

# gif(animation, "animation.gif", fps = 10)
# # display(animation)

In [8]:
# solvers = [Tsit5(), Vern7(), Midpoint()]
# results = Dict()

# for solver in solvers
#     @info "Benchmarking solver: $solver"
#     result = @btime solve($problem, $solver, save_everystep = false)
#     results[solver] = result
# end

# # Step 4: Collect Results

# # Step 5: Plotting
# plot()
# for (solver, result) in results
#     plot!(result, label=string(solver))
# end
# display(plot!)

In [9]:
function RHS!(ddu, du, u, p, t)
    A, sourcefunc, points, RightX, RightY = p
    fLX = sourcefunc(points, t, RightX, RightY)
    ddu .= -A*u .+ fLX
end

function run_simulation(Nx, Ny, tspan, coeffK, LeftX, LeftY, RightX, RightY, solver)
    Lx = RightX - LeftX
    Ly = RightY - LeftY
    hx = Lx/Nx
    hy = Ly/Ny

    A = create2DLFVM(Nx, Ny, hx, hy, coeffK, LeftX, LeftY, RightX, RightY)
    u0 = zeros((Nx-1)*(Ny-1), 1)
    du0 = u0
    points = grid(Nx, Ny, Lx, Ly)
    p = (; A, sourcefunc, points, RightX, RightY)

    problem = SecondOrderODEProblem(RHS!, du0, u0, tspan, p)
    solution = solve(problem, solver)
    
    return solution
end

tspan = (tStart, tEnd)
sizes = [(200, 100), (100, 50)]
solvers = [Tsit5(), RK4(), Midpoint()]



3-element Vector{OrdinaryDiffEq.OrdinaryDiffEqAdaptiveAlgorithm}:
 Tsit5(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false))
 RK4()
 Midpoint()

In [10]:
# for (Nx, Ny) in sizes
#     for solver in solvers
#         @info "Running simulation for Nx=$Nx, Ny=$Ny, solver=$solver"
#         try
#             @btime run_simulation($Nx, $Ny, $tspan, $coeffK, $LeftX, $LeftY, $RightX, $RightY, $solver)
#         catch e
#             @warn "Simulation failed for Nx=$Nx, Ny=$Ny, solver=$solver. Error: $e"
#         end
#     end
# end

In [11]:
function makeTable(benchmark::BenchmarkTools.Trial, Nx, Ny, solver)
    str = replace(string(solver), r"\([^)]*\)" => "")
    
    df = DataFrame(Algorithm = str)
    df.Nx .= Nx
    df.Ny .= Ny
    # df.Algorithm .= string(Tsit5())
    df.gctimes .= median(benchmark.gctimes)
    df.time .= median(benchmark.times)
    df.memory .= median(benchmark.memory)
    df.allocs .= median(benchmark.allocs)

    return df
end

makeTable (generic function with 1 method)

In [16]:
benchmark_results = Dict()
# combined_df = DataFrame()

for (Nx, Ny) in sizes
    for solver in solvers
        @info "Running simulation for Nx=$Nx, Ny=$Ny, solver=$solver"
        try
            benchmark = @benchmark run_simulation($Nx, $Ny, $tspan, $coeffK, $LeftX, $LeftY, $RightX, $RightY, $solver)
            benchmark_results[(Nx, Ny, solver)] = benchmark
            # df = makeTable(benchmark, Nx, Ny, solver)
            # append!(combined_df, df)
            @info "Time estimate: $(minimum(benchmark).time) seconds, Memory estimate: $(minimum(benchmark).memory) bytes"
        catch e
            @warn "Simulation failed for Nx=$Nx, Ny=$Ny, solver=$solver. Error: $e"
        end
    end
end

┌ Info: Running simulation for Nx=200, Ny=100, solver=Tsit5(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false))
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:6


┌ Info: Time estimate: 1.120871e9 seconds, Memory estimate: 1673986688 bytes
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:12
┌ Info: Running simulation for Nx=200, Ny=100, solver=RK4()
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:6


┌ Info: Time estimate: 1.1426136e9 seconds, Memory estimate: 1712768336 bytes
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:12


┌ Info: Running simulation for Nx=200, Ny=100, solver=Midpoint()
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:6
┌ Info: Time estimate: 5.178525e9 seconds, Memory estimate: 5466679456 bytes
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:12


┌ Info: Running simulation for Nx=100, Ny=50, solver=Tsit5(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false))
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:6
┌ Info: Time estimate: 1.656605e8 seconds, Memory estimate: 202048400 bytes
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:12
┌ Info: Running simulation for Nx=100, Ny=50, solver=RK4()
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:6


┌ Info: Time estimate: 3.393418e8 seconds, Memory estimate: 407899712 bytes
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:12
┌ Info: Running simulation for Nx=100, Ny=50, solver=Midpoint()
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:6


┌ Info: Time estimate: 1.1226305e9 seconds, Memory estimate: 1280907232 bytes
└ @ Main c:\Users\Admin\Desktop\Pattern-formation-in-sediment-transport-in-rivers\julia playground\wave.ipynb:12


In [13]:
combined_df 

UndefVarError: UndefVarError: `combined_df` not defined

In [18]:
benchmark_results[(100, 50, Tsit5())]

BenchmarkTools.Trial: 17 samples with 1 evaluation.
 Range (min … max):  165.661 ms … 526.742 ms  ┊ GC (min … max):  5.11% … 17.17%
 Time  (median):     257.595 ms               ┊ GC (median):    11.38%
 Time  (mean ± σ):   296.408 ms ± 108.484 ms  ┊ GC (mean ± σ):  12.98% ±  4.23%

  ▁▁ ▁  ▁    █▁▁ ▁         ▁ ▁▁ █      ▁                      █  
  ██▁█▁▁█▁▁▁▁███▁█▁▁▁▁▁▁▁▁▁█▁██▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  166 ms           Histogram: frequency by time          527 ms <

 Memory estimate: 192.69 MiB, allocs estimate: 6806.